In [ ]:
# INITIALIZE PYQGIS (run once per session)
import sys, os
os.environ['QT_QPA_PLATFORM']='offscreen'
sys.path.append('/apps/share64/debian7/anaconda/anaconda3-5.1/envs/qgis/')
from qgis.core import *
from qgis.analysis import QgsNativeAlgorithms
#from qgis.utils import *
import processing
from processing.core.Processing import Processing 

qgs = QgsApplication([], False)
qgs.initQgis()
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms()) 

In [ ]:
# IMPORT LIBRARIES
import ftplib, math, zipfile, shutil

In [ ]:
# USER INPUTS
input_folder_name = "/home/mygeohub/dey6/QGIS_Modules/DEM_Download/Class59100_B6/Input"
output_folder_name = "/home/mygeohub/dey6/QGIS_Modules/DEM_Download/Class59100_B6/Output" 
boundary_file =  "B6_simplify.shp"

In [ ]:
# USER DEFINED FUNCTION FOR DOWNLOADING TILES
def DownloadNED(lat,lon):
    name = "n"+lat+"w"+lon
    with ftplib.FTP('rockyftp.cr.usgs.gov') as ftp:
        try:
            ftp.login()
            ftp.cwd('vdelivery/Datasets/Staged/Elevation/1/ArcGrid/')
            contents = ftp.nlst()
            filtered_contents = [f for f in contents if ((name in f) & (".zip" in f))]
            if len(filtered_contents) == 0:
                print("No file found for: " + name)        
            elif len(filtered_contents) == 1:
                final_file = filtered_contents[0]
                print("1 file found for current tile")
                print("Downloading " + final_file)
                fo = open(os.path.join(work_folder_name, final_file), 'wb')
                ftp.retrbinary("RETR " + final_file , fo.write)
                fo.close() 
                print("Download successful")
                return(final_file)                
            elif len(filtered_contents) > 1:
                print("More than 1 file found: Dowloading largest zip file")
                file_list = []
                ftp.sendcmd("TYPE i")
                for f in filtered_contents:
                    file_list.append((f,ftp.size(f)))
                file_list.sort(key=lambda s: s[1])
                final_file = file_list[-1] #return the largest file
                print("Downloading..." + final_file)
                fo = open(os.path.join(work_folder_name, final_file), 'wb')
                ftp.retrbinary("RETR " + final_file , fo.write)
                fo.close()
                print("Download successful")
                return(final_file)
            else:
                print("Unknown error with file download for:" + name)           
            
        except ftplib.all_errors as e:
            print('FTP error:', e)
    

In [ ]:
# USER DEFINED FUNCTION FOR UNZIPPING DOWNLOADED TILES
def UnzipNED(f):
    try:
        zfile1 = zipfile.ZipFile(work_folder_name + "/" + f, 'r')
        zfile1.extractall(work_folder_name)
        zfile1.close()
        print("NED unzipped successfully: " + f)
    except:
        print("Error in unzipping: " + f)
    
    

In [ ]:
# MAIN CODE STARTS
work_folder_name = os.path.join(input_folder_name, "WorkFolder")
if os.path.exists(work_folder_name) == False:
    os.mkdir(work_folder_name)
boundary_path = os.path.join(input_folder_name, boundary_file)
input_crs = QgsVectorLayer(boundary_path, '', 'ogr' ).crs().authid()
#processing.run('qgis:reprojectlayer',{'INPUT': full_input_path, 'TARGET_CRS':'EPSG:102673','OUTPUT': folder_name + "boundary_proj.shp"})
processing.run('native:reprojectlayer',{'INPUT': boundary_path, 'TARGET_CRS':'EPSG:4326','OUTPUT': work_folder_name + "/boundary_proj.shp"})

ext = QgsVectorLayer(work_folder_name + "/boundary_proj.shp", '', 'ogr' ).extent()
# print the extents here (Exercise 1)

# generate list of intermediate latitudes and longitudes (Exercise 1)
listNL= [] # list of intermediate latitudes
listWL= [] # list of intermediate longtiudes


In [ ]:
# Looping through tiles to download
raster_names = [] # this is used to store the name of the rasters downloaded
for i in listNL:
    for j in listWL:
        # create name string for current pair of lat-long
        # call DownloadNED
        # call UnzipNED
        raster_names.append("grdn"+lat+"w"+lon + "_1") # storing names of downloaded rasters
print("Tile processing complete...")

In [ ]:
#Clean Up (run if you want to shut down qgis)
qgs.exitQgis()